In [ ]:
# first, install holoviews package through the following command line: 
# conda install -c ioam holoviews bokeh

# then, initiate this notebook with:
# jupyter notebook --NotebookApp.iopub_data_rate_limit=10000000000

# ----------------
# compute and plot the dynamic functional connectivity with use of holoviews package:
# http://holoviews.org/getting_started/index.html
import holoviews as hv
import numpy as np
hv.extension('bokeh')

import pandas as pd
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
from sklearn.covariance import GraphLassoCV
from sklearn.metrics import mutual_info_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
from __future__ import division

In [ ]:
# load the fMRI data:
df1             = pd.read_csv("100307_restingstate_day1.csv")
data1           = df1.values
data1           = df1.values[0:600,:] #(I need to shorten the dataset a little bt as it is too large for plotting dFC)
Nvars           = data1.shape[1]
T               = data1.shape[0]
labels_Glasser  = pd.read_csv("labels_Glasser.csv",header=None)[0].tolist()

In [ ]:
# define the window size:
window_length   = 500 # define window length as a number or as a fraction of the data length

In [ ]:
# [01] dynamic functional connectivity through a sliding window:
Nwindows = T - window_length + 1
dynamic_corrs1 = np.zeros((Nwindows,Nvars,Nvars))
for ind in range(Nwindows):
    frame = data1[ind:ind + window_length, :]   
    dynamic_corrs1[ind, :, :] = np.corrcoef(np.transpose(frame))

In [ ]:
# create tuple of tuples with labels:
labels = ()
for ind in range(Nvars):
    labels = labels + ((ind,labels_Glasser[ind]),)
  
xticks = labels
yticks = labels

In [ ]:
# create a 3D interactive map of FC matrices with one slider (window number):
%opts Image (cmap='RdBu_r') [colorbar=True invert_yaxis=False xticks=xticks yticks=yticks xrotation=90]
items_fc = {} 
for n in range(Nwindows): # a slider: window number
        def get_image3D(mat, label, n):
            mat = mat[n, :, :]
            return hv.Image(mat, label=label, bounds=[0,0,Nvars,Nvars],
                 kdims=['ROIs', 'ROIs'], 
                 vdims=[hv.Dimension('bla', range=(-0.5, 0.5))])

        image_frame = get_image3D(dynamic_corrs1, 'dynamic FC, rectangular window, ', n)
        items_fc[n] = image_frame  
        
map1 = hv.HoloMap(items_fc, kdims=['window number'])

In [ ]:
# plot the output:
%output size=250 max_frames=200 
map1